In [1]:
import datetime
import time

import pandas as pd
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pgpasslib

from querymsc import WorkflowTools
%load_ext autoreload
%autoreload 2

In [2]:
connection_string = "postgresql+psycopg2://nannau@dbmsc.pcic.uvic.ca/msc"
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
start_time = datetime.datetime(1990, 1, 1)
end_time = datetime.datetime(1992, 1, 1)

msc = WorkflowTools(start_time, end_time)

2.0


In [4]:
annual_tot_precip = msc.query_annual_precip(session)
annual_tot_rain = msc.query_annual_rain(session)
query_temp = msc.query_design_temp_percentile(session)

In [7]:
query = msc.query_design_temp_percentile(session)

In [6]:
annual_tot_precip = msc.query_annual_precip(session)
pd.read_sql(annual_tot_precip.statement, engine)

,annual_precip,time_min,time_max,lat,lon,station_id,obs_count,completeness
0,932.688390,1990-01-01,1991-12-31,41.7833,-82.6833,6063,730,1.000000
1,966.911815,1990-01-01,1991-12-31,41.9500,-82.5167,6140,730,1.000000
2,995.731541,1990-01-01,1991-12-31,42.0333,-82.9000,6036,730,1.000000
3,1011.392260,1990-01-01,1991-12-31,42.0431,-82.6739,6040,730,1.000000
4,959.106473,1990-01-01,1991-12-31,42.1034,-83.0945,5992,730,1.000000
5,583.599452,1990-01-01,1991-12-31,42.2000,-82.2000,6071,684,0.936986
6,889.759007,1990-01-01,1991-12-31,42.2167,-82.7333,6137,730,1.000000
7,1015.595137,1990-01-01,1991-12-31,42.2756,-82.9556,6130,730,1.000000
8,825.715171,1990-01-01,1991-12-31,42.3333,-83.0167,6132,610,0.835616
9,899.915959,1990-01-01,1991-12-31,42.3900,-82.2153,6008,727,0.995890


In [7]:
annual_tot_rain = msc.query_annual_rain(session)
pd.read_sql(annual_tot_rain.statement, engine)

,annual_rain,time_min,time_max,lat,lon,station_id,completeness
0,895.713082,1990-01-01,1991-12-31,41.7833,-82.6833,6063,1.000000
1,906.670582,1990-01-01,1991-12-31,41.9500,-82.5167,6140,1.000000
2,922.481404,1990-01-01,1991-12-31,42.0333,-82.9000,6036,1.000000
3,962.008459,1990-01-01,1991-12-31,42.0431,-82.6739,6040,1.000000
4,886.156541,1990-01-01,1991-12-31,42.1034,-83.0945,5992,1.000000
5,466.819521,1990-01-01,1991-12-31,42.2000,-82.2000,6071,0.821918
6,848.180548,1990-01-01,1991-12-31,42.2167,-82.7333,6137,1.000000
7,928.435479,1990-01-01,1991-12-31,42.2756,-82.9556,6130,1.000000
8,665.155274,1990-01-01,1991-12-31,42.3333,-83.0167,6132,0.710959
9,823.563699,1990-01-01,1991-12-31,42.3900,-82.2153,6008,0.957534


In [5]:
query_temp = msc.query_design_temp_percentile(session)
pd.read_sql(query_temp.statement, engine)

,air_temperature,time_min,time_max,lat,lon,station_id,completeness
0,-17.630,1990-01-01,1991-01-31,41.7833,-82.6833,6063,0.951613
1,-16.100,1990-01-01,1991-01-31,41.9500,-82.5167,6140,0.983871
2,-15.900,1990-01-01,1991-01-31,42.0333,-82.9000,6036,0.983871
3,-16.500,1990-01-01,1991-01-31,42.0431,-82.6739,6040,0.967742
4,-15.000,1990-01-01,1991-01-31,42.1034,-83.0945,5992,0.935484
5,-15.690,1990-01-01,1991-01-31,42.2000,-82.2000,6071,0.887097
6,-15.646,1990-01-01,1991-01-31,42.2167,-82.7333,6137,1.000000
7,-14.473,1990-01-01,1991-01-31,42.2756,-82.9556,6130,1.000000
8,-14.400,1990-01-01,1991-01-31,42.3333,-83.0167,6132,0.983871
9,-9.500,1990-01-01,1990-01-31,42.3900,-82.2153,6008,0.451613


In [6]:
query_design_temp_dry = msc.query_design_temp_dry(session)
pd.read_sql(query_design_temp_dry.statement, engine).head()

,wet_bulb_temperature,time_min,time_max,lat,lon,station_id,completeness
0,20.0000,1990-07-01,1991-07-31,41.7833,-82.6833,6063,1.000000
1,19.7625,1990-07-01,1991-07-31,41.9500,-82.5167,6140,1.000000
2,19.5250,1990-07-01,1991-07-31,42.0333,-82.9000,6036,1.000000
3,19.4750,1990-07-01,1991-07-31,42.0431,-82.6739,6040,0.967742
4,20.7625,1990-07-01,1991-07-31,42.1034,-83.0945,5992,1.000000


In [9]:
query_design_temp_wet = msc.query_design_temp_wet(session)
pd.read_sql(query_design_temp_wet.statement, engine)

62.0


,wet_bulb_temp,time_min,time_max,lat,lon,station_id,completeness,obs_count
0,12.7175,1990-07-01 00:00:00,1991-07-31 23:00:00,42.2756,-82.9556,6130,1.000000,1488
1,12.3100,1990-07-01 10:00:00,1991-07-31 20:00:00,42.9922,-82.3047,5970,0.433468,645
2,10.7125,1990-07-01 00:00:00,1991-07-31 23:00:00,43.0331,-81.1511,6216,0.998656,1486
3,11.7775,1990-07-01 00:00:00,1991-07-31 23:00:00,43.1717,-79.9342,6373,0.666667,992
4,12.8000,1990-07-01 11:00:00,1991-07-31 21:00:00,43.2000,-79.1667,6088,0.421371,627
5,10.0350,1990-07-01 10:00:00,1991-07-31 20:00:00,43.4500,-80.3833,6266,0.435484,648
6,12.7550,1990-07-01 00:00:00,1991-07-31 23:00:00,43.6286,-79.3950,6582,0.687500,1023
7,10.6175,1990-07-01 00:00:00,1991-07-31 23:00:00,43.6772,-79.6306,6592,1.000000,1488
8,13.3050,1990-07-01 00:00:00,1991-07-31 00:00:00,43.7667,-81.7167,5940,1.000000,62
9,9.7000,1990-07-01 00:00:00,1991-07-31 23:00:00,43.8308,-66.0886,8348,1.000000,1488


In [10]:
hdd = msc.query_deg_day_below_18(session)
pd.read_sql(hdd.statement, engine)

,hdd,time_min,time_max,lat,lon,station_id,completeness
0,2408.65,1990-01-01,1991-12-31,41.7833,-82.6833,6063,0.991781
1,2674.05,1990-01-01,1991-12-31,41.9500,-82.5167,6140,0.989041
2,2623.50,1990-01-01,1991-12-31,42.0333,-82.9000,6036,0.984932
3,2698.20,1990-01-01,1991-12-31,42.0431,-82.6739,6040,0.964384
4,2490.50,1990-01-01,1991-12-31,42.1034,-83.0945,5992,0.989041
5,2412.50,1990-01-01,1991-12-31,42.2000,-82.2000,6071,0.815068
6,2712.75,1990-01-01,1991-12-31,42.2167,-82.7333,6137,0.998630
7,2691.40,1990-01-01,1991-12-31,42.2756,-82.9556,6130,0.997260
8,1562.70,1990-01-01,1991-12-31,42.3333,-83.0167,6132,0.671233
9,2328.40,1990-01-01,1991-12-31,42.3900,-82.2153,6008,0.953425


In [29]:
df_vars = pd.read_sql(msc.query_variables(session).statement, engine)
#df_vars[df_vars['net_var_name']=='127']
#df_vars[(df_vars['net_var_name'].astype(int) > 124) & (df_vars['net_var_name'].astype(int) < 132)]
df_vars[df_vars['standard_name'].str.contains('rain')]

,vars_id,standard_name,long_description,unit,cell_method,net_var_name
91,1380,thickness_of_rainfall_amount,Hourly rainfall,0.1 mm,time: sum,123
93,1382,rainfall_rate,5 minutes,0.1mm,,125
94,1383,rainfall_rate,10 minutes,0.1mm,,126
95,1384,rainfall_rate,15 minutes,0.1mm,,127
96,1385,rainfall_rate,30 minutes,0.1mm,,128
97,1386,rainfall_rate,1 hour,0.1mm,,129
98,1387,rainfall_rate,2 hours,0.1mm,,130
99,1388,rainfall_rate,6 hours,0.1mm,,131
100,1389,rainfall_rate,12 hours,0.1mm,,132
102,1391,rainfall_rate,24 hours,0.1mm,,161


In [12]:
rain_rate = msc.query_rain_rate_15(session)
df_rain_rate = pd.read_sql(rain_rate.statement, engine)
df_rain_rate[df_rain_rate['completeness'] > 0.6]

,rainfall_rate,time_min,time_max,lat,lon,station_id,completeness
1,18.8,1990-01-01,1991-12-31,42.2756,-82.9556,6130,0.968493
4,16.9,1990-04-03,1991-11-30,42.8667,-80.5500,6013,0.656164
7,18.6,1990-01-01,1991-12-31,43.0331,-81.1511,6216,0.938356
11,31.1,1990-01-01,1991-11-30,43.1717,-79.9342,6373,0.932877
12,17.2,1990-02-02,1991-12-31,43.2000,-79.1667,6088,0.819178
13,14.4,1990-04-01,1991-11-30,43.2833,-79.8833,6378,0.612329
19,14.0,1990-04-01,1991-10-31,43.6286,-79.3950,6582,0.658904
25,16.5,1990-01-01,1991-12-31,43.6772,-79.6306,6592,0.987671
28,14.9,1990-01-01,1991-12-31,43.8308,-66.0886,8348,0.982192
31,14.2,1990-01-01,1991-12-31,43.8667,-66.1000,8104,0.980822


In [19]:
df_rain_rate['obs_count'].value_counts().idxmax()

55